# DYR Functions

In [1]:
import numpy as np
import pandas as pd

from typing import Dict

import andes

In [2]:
%matplotlib inline

In [3]:
andes.config_logger(stream_level=20)

In [ ]:
def write_dict_dyr(dyr_dict: Dict[str, pd.DataFrame], outfile: str):
    """
    Write a dictionary of dynamic models to a DYR file.

    This function takes a dictionary where the keys are model names (e.g., 'REGCA1', 'REECA1') 
    and the values are pandas DataFrames containing the model parameters. It writes the data 
    to a DYR file in the correct format, ensuring the following:
    - The first column (IBUS) is an integer representing the bus index.
    - The second column is the model name, enclosed in single quotes.
    - The third column (ID) is an integer representing the device index.
    - Remaining columns are the model parameters, written as space-separated values.
    - Each line ends with a trailing slash ('/').

    Models 'Toggle' are skipped during the writing process.

    Parameters:
    -----------
    dyr_dict : Dict[str, pd.DataFrame]
        A dictionary where each key is a model name and each value is a pandas DataFrame 
        containing the parameters for that model. The DataFrame must have the following structure:
        - Column 0: IBUS (integer)
        - Column 1: ID (integer)
        - Columns 2+: Model parameters (various types).
    
    outfile : str
        The path to the output DYR file where the formatted data will be written.

    Returns:
    --------
    None
        The function writes the data to the specified file and prints a success message.

    Notes:
    ------
    - The function skips models with the name 'Toggle' and prints a message for each skipped model.
    - A blank line is not added between models in the output file.
    - Ensure the input DataFrames are properly formatted to avoid runtime errors.

    Example:
    --------
    >>> dyr_dict = {
    ...     'REGCA1': pd.DataFrame([
    ...         [1004, 1, 1.0, 0.01, 10, 0.9, 0.5],
    ...         [1006, 1, 1.0, 0.02, 10, 0.9, 0.5]
    ...     ]),
    ...     'REECA1': pd.DataFrame([
    ...         [1004, 1, 0, 0, 1, 1, 1],
    ...         [1006, 1, 0, 0, 1, 1, 1]
    ...     ])
    ... }
    >>> write_dict_dyr(dyr_dict, './output.dyr')
    Data successfully written to ./output.dyr
    """
    with open(outfile, 'w') as f:
        for model_name, df in dyr_dict.items():
            if model_name in ['Toggle']:
                print(f"Skipped {model_name}")
                continue
            for _, row in df.iterrows():
                ibus = int(row.iloc[0])  # format IBUS as integer
                model = f"'{model_name}'"  # model name
                device_id = int(row.iloc[1])  # format ID as integer
                # format the rest parameters
                params = ' '.join(map(str, row.iloc[2:].values))
                # Combine all parts into a formatted line
                formatted_line = f"{ibus} {model} {device_id} {params} /"
                # Write the formatted line to the file
                f.write(formatted_line + '\n')

    print(f"Data successfully written to {outfile}")

Skipped Toggle
Data successfully written to ./Texas2k/ieee14_out.dyr


In [ ]:
# Test on IEEE 14 case
dyr = andes.io.psse._read_dyr_dict("./Texas2k/ieee14.dyr")
write_dict_dyr(dyr, './Texas2k/ieee14_out.dyr')

In [7]:
sa1 = andes.load(
    "./Texas2k/ieee14.raw",
    addfile="./Texas2k/ieee14.dyr"
)

Working directory: "/Users/jinningwang/work/turbinegov"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Parsing input file "./Texas2k/ieee14.raw"...
  IEEE 14 BUS TEST CASE
  03/06/14 CONTO           100.0  1962 W
Input file parsed in 0.0117 seconds.
Parsing additional file "./Texas2k/ieee14.dyr"...
Addfile parsed in 0.0688 seconds.
IEEEST <IEEEST_1> added BusFreq <BusFreq_1> linked to bus <3.0>
ST2CUT <ST2CUT_2> added BusFreq <BusFreq_2> linked to bus <1.0>
ST2CUT <ST2CUT_3> added BusFreq <BusFreq_3> linked to bus <2.0>
System internal structure set up in 0.0610 seconds.


In [8]:
sa2 = andes.load(
    "./Texas2k/ieee14.raw",
    addfile="./Texas2k/ieee14_out.dyr"
)

Working directory: "/Users/jinningwang/work/turbinegov"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Reloaded generated Python code of module "pycode".
Parsing input file "./Texas2k/ieee14.raw"...
  IEEE 14 BUS TEST CASE
  03/06/14 CONTO           100.0  1962 W
Input file parsed in 0.0051 seconds.
Parsing additional file "./Texas2k/ieee14_out.dyr"...
Addfile parsed in 0.0853 seconds.
IEEEST <IEEEST_1> added BusFreq <BusFreq_1> linked to bus <3.0>
ST2CUT <ST2CUT_2> added BusFreq <BusFreq_2> linked to bus <1.0>
ST2CUT <ST2CUT_3> added BusFreq <BusFreq_3> linked to bus <2.0>
System internal structure set up in 0.0147 seconds.
